# Social Networks - Assignment 4

This **Home Assignment** is **mandatory**, it does count towards your final chance to take part in the exam. 

You can expect numpy, torch, torch_geometric, matplotlib, and networkx to be installed.

## Formalities

**Submit in a group of 3-4 people until 10.07.2022 23:59CET. The deadline is strict!**

You have the opportunity for an prelim submission until 07.07.2022 23:59 CET. We will run your code once, and you can thus see whether there are small bugs that you might not have expected.

## Evaluation and Grading
General advice for programming excercises at *CSSH*:
Evaluation of your submission is done semi-automatically. Think of it as this notebook being 
executed once. Afterwards, some test functions are appended to this file and executed respectively.

Therefore:
* Submit valid _Python3_ code only!
* Use external libraries only when specified by task.
* Ensure your definitions (functions, classes, methods, variables) follow the specification if
  given. The concrete signature of e.g. a function usually can be inferred from task description, 
  code skeletons and test cases.
* Ensure the notebook does not rely on current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
* Keep your code idempotent! Running it or parts of it multiple times must not yield different
  results. Minimize usage of global variables.
* Ensure your code / notebook terminates in reasonable time.

**There's a story behind each of these points! Don't expect us to fix your stuff!**

Regarding the scores, you will get no points for a task if:
- your function throws an unexpected error (e.g. takes the wrong number of arguments)
- gets stuck in an infinite loop
- takes much much longer than expected (e.g. >1s to compute the mean of two numbers)
- does not produce the desired output (e.g. returns an descendingly sorted list even though we asked for ascending, returns the mean and the std even though we asked for only the mean, prints an output instead of returning it!)

In [ ]:
# credentials of all team members (you may add or remove members from the list)
team_members = [
    {
        'first_name': 'Some',
        'last_name': 'One',
        'student_id': 123451
    },
    {
        'first_name': 'Bob',
        'last_name': 'Bar',
        'student_id': 54321
    },
    {
        'first_name': 'Not',
        'last_name': 'Sure',
        'student_id': 15432
    }
]

In [3]:
!pip install torch

In [2]:
# As usual all provided output was obtained with the following torch version
#  different outputs could be possible with different package version
import torch
print(torch.__version__) #1.10.1
import torch_geometric
print(torch_geometric.__version__) # 2.0.4

ModuleNotFoundError: No module named 'torch'

## You own GCN (5)

Create a GCN layer as introduced in the lecture (compare deck 09 slide 62, without the activation function) using PyTorch Geometric. Therefore, use the `MyGCN` class provided below and add code to the `forward` and `message` functions. 

While in the lecture the normalization constant $c_i$ is $|N(i)|$, for this task we are using a normalization function of $c_i = \sqrt{|N_{\text{in}}(i)| |N_{\text{out}}(i)|}$ which for undirected networks comes out to be equal to the lecture definition.

Notice that datasets that we encounter later on are (potentially) directed.

In [ ]:
import torch
from torch.nn import Linear, Parameter
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree

class MyGCN(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')
        
        
        self.W0 = Linear(in_channels, out_channels, bias=False)
        self.W1 = Linear(in_channels, out_channels, bias=False)
        self.b = Parameter(torch.Tensor(out_channels))

        self.reset_parameters()
        
    def reset_parameters(self):
        self.W0.reset_parameters()
        self.W1.reset_parameters()
        self.b.data.zero_()
        
    def forward(self, x, edge_index):
        pass
    
        """Implement most of the work here
        Notice that pytorch geometric MessagePassing class inherits self.propagate
          which essentially already has most of the message passing implemented already
        """
    
    def message(self, x_j, norm):
        """Implement normalisation here"""
        pass

### Create example Graph (deck 09 slide 64)

In [ ]:
import networkx as nx

In [ ]:
s = """0 1
0 2
0 3
1 0
1 2
1 3
2 0
2 1
2 4
3 0
3 1
3 4
4 2
4 3
4 5
5 4
5 6
5 7
6 5
6 8
6 9
7 5
7 8
7 9
8 6
6 7
8 9
9 7
9 6
9 8"""

In [ ]:
edges = []
for line in s.splitlines():
    a, b = line.split(" ")
    edges.append((int(a),int(b)))

In [ ]:
G = nx.Graph(edges)
my_pos = nx.spring_layout(G, seed = 4)
nx.draw(G, pos=my_pos)

### Reproduce the lecture results (2)

In the lecture, you have seen one special case of a GCN (on deck 09 slides 64 you find the definition). To reproduce the results of this case here, we need two functions `create_initial_embeddings` (returning the initial embeddings as specified on the lecture slide and `set_weights` (adjusting the weights of the GCN to be equal to those of the lecture slide, no output). Implement both functions below.


In [ ]:
def create_initial_embeddings():
    """Your code here"""
    return torch.tensor([(0,1)],dtype=torch.float)

In [ ]:
initial_embed = create_initial_embeddings()
initial_embed.shape #torch.Size([10, 2])

In [ ]:
def set_weights(layer):
    with torch.no_grad():
        """Your code here"""

In [ ]:
# Test
layer = MyGCN(2,2)
set_weights(layer)

In [ ]:
layer.W0.weight

#Parameter containing:
#tensor([[1., 0.],
#        [0., 0.]], requires_grad=True)

In [ ]:
layer.W1.weight

#Parameter containing:
#tensor([[0., 0.],
#        [1., 0.]], requires_grad=True)

In [ ]:
layer.b

#Parameter containing:
#tensor([0., 0.], requires_grad=True)

In [ ]:
edge_index = torch.tensor(edges, dtype=torch.long).T
edge_index

#tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7,
#         8, 6, 8, 9, 9, 9],
#        [1, 2, 3, 0, 2, 3, 0, 1, 4, 0, 1, 4, 2, 3, 5, 4, 6, 7, 5, 8, 9, 5, 8, 9,
#         6, 7, 9, 7, 6, 8]])

In [ ]:
layer(initial_embed, edge_index)
#tensor([[0.0000, 0.0000],
#        [0.0000, 0.0000],
#        [0.0000, 0.3333],
#        [0.0000, 0.3333],
#        [1.0000, 0.3333],
#        [1.0000, 0.3333],
#        [0.0000, 0.3333],
#        [0.0000, 0.3333],
#        [0.0000, 0.0000],
#        [0.0000, 0.0000]], grad_fn=<AddBackward0>)

## Multi Layer GCN (3)

We will now use our defined Convolution Layer to create a GCN with variable number of "hidden layers" using the `MultiLayerGCN` class below.

The network always starts with an initial layer mapping node features onto the size of the hidden dim. No activation function is used after this layer. This layer is of shape `node_features_dim` x `hidden_dim`.
Then, we apply `num_hidden_layers` hidden layers of shape `hidden_dim` x `hidden_dim`, each followed by a `ReLu` activation function.
The network terminates with an output layer of shape `hidden_dim ` x `num_classes`  that is followed by a `log_sofmax` activation.

All layers are of type `MyGCN`. For reproducability reasons, layers are initialized in the same order they are traversed in a forward pass. You find a usage example below.

In [ ]:
from torch_geometric.datasets import Planetoid

In [ ]:
class MultiLayerGCN(torch.nn.Module):
    def __init__(self, node_features_dim, hidden_dim, num_classes, num_hidden_layers=0):
        super().__init__()
        self.layers = []
        """Your code to fill up the layers here"""
        
        self.layers = torch.nn.ModuleList(self.layers)

    def forward(self, data):
        """Your code for the forward pass here"""
        pass

In [ ]:
# example training loop
def training(model, data, n_epochs = 200):

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    model.train()
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

In [ ]:
# checking initialization with fixed seed
torch.manual_seed(1)
tmp_GCN = MultiLayerGCN(3, 2, 2, 0)
print(tmp_GCN.layers[0].W0.weight)
#tensor([[ 0.2109, -0.2250, -0.0421],
#        [-0.0520,  0.0837, -0.0023]], requires_grad=True)
tmp_GCN.layers[1].W0.weight
#tensor([[ 0.4667, -0.6443],
#        [-0.6723, -0.3411]], requires_grad=True)

In [ ]:
# checking initialization with fixed seed
torch.manual_seed(1)
tmp_GCN = MultiLayerGCN(3, 2, 2, 1)
for i in range(3):
    print(tmp_GCN.layers[i].W0.weight)
    
# Parameter containing:
#tensor([[ 0.2109, -0.2250, -0.0421],
#        [-0.0520,  0.0837, -0.0023]], requires_grad=True)
#Parameter containing:
#tensor([[ 0.4667, -0.6443],
#        [-0.6723, -0.3411]], requires_grad=True)
#Parameter containing:
#tensor([[-0.2042, -0.0775],
#        [-0.6798, -0.3371]], requires_grad=True)


#### Import the Cora dataset

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora', split="public")

In [ ]:
torch.manual_seed(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiLayerGCN(dataset.num_node_features, 16, dataset.num_classes, 0).to(device)


data = dataset[0].to(device)

training(model, data)

#### Evaluate the model on the test set

In [ ]:
def get_test_accuracy(model, data):
    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

In [ ]:
acc = get_test_accuracy(model, data)
print(f'# Accuracy: {acc:.8f}')
# Accuracy: 0.77000000

### Are more layers always better? (3)

You will now look into finding the optimal number of layers that you should use in your Multi-Layer GCN. Therefore, evaluate different configurations on different datasets. For the three datasets `("Cora", "CiteSeer", "PubMed")` compute the accuracy of a fitted `MultiLayerGCN` with `hidden_dim` of 16 and a varying number (`[0, 1, 2, 3, 4]`) of hidden layers.

Repeat the experiment 10 times for each of the settings using a different seed ([0..9]) each time.

Collect the result of the experiments in a dictionary `results` that maps the names of the datasets to list of list. e.g. `results["Cora"][1][2]` holds the results for the Cora dataset with 1 "hidden layer" and an initial seed of 2.

Use the `"public"` splits of the datasets. Plot the mean and standard deviation of the accuracy as a function of hidden layers in an errorbar plot. Save the a plot containing lines for all three datasets as `accuracies.png` here in the notebook.

### What do we observe? (2)

Describe what you see in your own words. Then interpret your findings. Save your text in the `your_observation` variable provided below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F

In [ ]:
# computations take about 5 min
results = {"Cora" : [], "CiteSeer" : [], "PubMed" : []}
for dataset_name in ("Cora", "CiteSeer", "PubMed"):
    for n_hidden in range(5):
        for seed in range(10):
            torch.manual_seed(seed) # !set seed before initializing the model!

In [ ]:
results
#{'Cora': 
# [[0.767,   0.77,   0.778,   0.777,   0.764,   0.777,   0.773,   0.762,   0.763,   0.774],
#  [0.792, 0.785, 0.777, 0.776, 0.78, 0.776, 0.769, 0.789, 0.799, 0.778],
#  [0.747, 0.76, 0.745, 0.769, 0.747, 0.736, 0.776, 0.781, 0.761, 0.766],
#  [0.727, 0.737, 0.729, 0.688, 0.729, 0.744, 0.727, 0.717, 0.593, 0.709],
#  [0.687, 0.598, 0.674, 0.702, 0.716, 0.746, 0.679, 0.651, 0.719, 0.738]],
# 'CiteSeer': 
# [[0.694,   0.688,   0.686,   0.69,   0.681,   0.688,   0.679,   0.679,   0.685,   0.687],
#  [0.674, 0.68, 0.679, 0.643, 0.675, 0.687, 0.665, 0.67, 0.638, 0.663],
#  [0.59, 0.576, 0.641, 0.624, 0.598, 0.568, 0.637, 0.61, 0.655, 0.55],
#  [0.627, 0.519, 0.625, 0.608, 0.627, 0.631, 0.55, 0.612, 0.5, 0.563],
#  [0.607, 0.524, 0.521, 0.476, 0.491, 0.576, 0.47, 0.509, 0.543, 0.578]],
# 'PubMed':
# [[0.77,   0.769,   0.759,   0.767,   0.77,   0.769,   0.764,   0.769,   0.766,   0.768],
#  [0.79, 0.774, 0.779, 0.786, 0.788, 0.788, 0.784, 0.786, 0.788, 0.787],
#  [0.763, 0.792, 0.793, 0.782, 0.786, 0.751, 0.623, 0.797, 0.772, 0.783],
#  [0.786, 0.734, 0.723, 0.764, 0.779, 0.778, 0.714, 0.704, 0.735, 0.756],
#  [0.668, 0.667, 0.767, 0.759, 0.67, 0.725, 0.707, 0.724, 0.771, 0.728]]}

In [ ]:
your_observation = "Here goes your text"